# Product Requirements Document (PRD)

**Project**: LEGO Lean Production Simulation  
**Version**: V0.1  
**Owner**: Yicheng Fan  
**Last Updated**: 2025-11-04  

---

## 1. Project Background

To enhance understanding and practical application of Lean Production principles, this simulation system is built using LEGO as a physical abstraction. Although not based on a real-world factory floor, by decomposing the production flow, assigning teams, and simulating material and information flow, we aim to replicate key variables and types of waste (e.g., waiting, transportation, inventory) in manufacturing and use the simulation for performance assessment and continuous improvement.

---

## 2. Objectives

- Build an abstract but functional model of a lean production system  
- Implement a simulation engine using Python (or R) with interactive execution  
- Allow configurable inputs and random events to test various lean strategies  
- Support iterative experiments for comparison and optimization  

---

## 3. Conceptual Modeling

### 3.1 Team Assignments

- Team 1: Type Sorting  
- Team 2: Set Sorting  
- Team 3: Axis Assembly  
- Team 4: Chassis Assembly  
- Team 5: Final Assembly  

### 3.2 Resource Objects

- Raw materials (Bricks)  
- Intermediate warehouses (B, C1, C2, C3, D1, D2)  
- Finished goods warehouse (E)  
- Tasks/Orders (Gliders)  

### 3.3 Model Elements

- **State Variables**: inventory levels, queue lengths, completed items, simulation clock  
- **Parameters**: average process time, team size, transport time, shift length, initial stock  
- **Random Events**: missing bricks, rework, transport delay, human error  
- **Assumptions**: FIFO processing, fixed initial inventory, one task per worker, one task per team per round  

---

## 4. Core Functional Requirements

### 4.1 Simulation Engine

- Event-driven or time-stepped processing  
- Each team action modeled as a processing node  
- Support for both step-by-step and auto-run modes  

### 4.2 Parameter Configuration

- User-defined process times (constant or distribution-based)  
- Team size, buffer size, initial inventory, task generation pace  

### 4.3 Visualization & Interaction

- Show real-time buffer status, queues per node  
- Live KPI dashboard  
- Pause, reset, and playback capabilities  

### 4.4 Random Event Simulation

- Probability-based disruptions (e.g., missing parts, rework, blocking)  
- Log all exceptions and reflect impact on KPIs  

---

## 5. Non-Functional Requirements

- Response time ≤ 1s per step  
- Local single-user deployment (CLI or simple Web UI)  
- Modular model structure for future expansion  
- Use `pandas` for data tracking, `matplotlib`/`plotly` for visualization  

---

## 6. KPI Design and Tracking

| KPI Name        | Description                          | Calculation Method                             |
|-----------------|--------------------------------------|-------------------------------------------------|
| Throughput      | Completed gliders per unit time      | Total completed / simulation time              |
| Lead Time       | Avg time per glider from start to end| Completion time - start time (average)         |
| WIP Level       | Items being processed                | Time-weighted average count of in-process items|
| Inventory Level | Items in buffers                     | Periodic snapshot of each warehouse stock      |
| Utilization     | Resource usage rate                  | Busy time / total time per team                |
| Error Rate      | Rate of disruptions                  | Exception count / total operations             |

---

## 7. Data Interface (Initial)

| Name          | Type   | Example | Description                            |
|---------------|--------|---------|----------------------------------------|
| process_time  | float  | 2.5     | Avg process time per task (sec)        |
| team_size     | int    | 3       | Number of people per team              |
| buffer_limit  | int    | 10      | Max warehouse capacity                 |
| shift_length  | int    | 480     | Shift length in minutes                |
| error_rate    | float  | 0.05    | Probability of a random disruption     |

---

## 8. Next Steps

1. Await detailed configuration: team setup, process times, error probabilities  
2. Build initial Python discrete-event simulation script (with logging and basic visualization)  
3. Deliver first working prototype in 2 weeks with minimum closed-loop simulation and KPI output (Front-end needed)

# LEGO Lean Simulation – Environment Definition

## Overview
This repository defines the **simulation environment** for the LEGO Lean Production project.
The goal of the simulation is to model the full process flow of assembling LEGO ski gliders, from raw bricks to sold products, following lean manufacturing principles.
This environment serves as the foundation for the mathematical model and Python simulation that will be developed in later sprints.

---

## Purpose
The `environment.py` file (converted from the Excel sheets) will store all parameters, object definitions, and process logic required for the simulation.
It replaces the previous Excel environment tables (`Parameters`, `Objects`, and `Stations`) with Python-friendly data structures.

---

## Environment Structure

### 1. Global Parameters

| Parameter | Value | Description |
|------------|--------|-------------|
| Number of glider types | 4 types | Color variants only (no structural difference) |
| Bricks per glider | 44 bricks | Total per finished glider |
| Bricks per Axis | 13 bricks | Used by Team 3 |
| Bricks per Chassis | 21 bricks | Used by Team 4 |
| Bricks for Final Assembly | 10 bricks | Used by Team 5 |
| Cost per brick | €100.00 | Material cost |
| Selling price per glider | €10,000.00 | Sales revenue |
| Profit margin per glider | €5,600.00 | Derived (10,000 - 44 × 100) |

---

### 2. Objects and Initial Conditions

| Object | Initial Number | Description |
|---------|----------------|-------------|
| Target output (N) | 8 | Planned gliders to produce |
| Bricks (A) | 352 | Raw input (44 × 8) |
| Sorted Bricks (B) | 0 | Output of Team 1 |
| Axis Set (C1) | 0 | Output of Team 2 |
| Chassis Set (C2) | 0 | Output of Team 2 |
| Final Assembly Set (C3) | 0 | Output of Team 2 |
| Finished Axes (D1) | 0 | Output of Team 3 |
| Finished Chassis (D2) | 0 | Output of Team 4 |
| Finished Gliders (E) | 0 | Output of Team 5 |
| Sold Gliders (F) | 0 | Output of Team 6 (end of process) |

---

### 3. Station Setup

| Station | Task | Inflow | Outflow | Work Time (sec) | Delivering Time (sec) | Cycle Time (sec) | Batch Size | Queue Capacity | Notes |
|----------|------|---------|----------|------------------|------------------------|------------------|-------------|----------------|-------|
| Team 1 – Type Sorting | Sort bricks by type and color | A | B | 44 | 10 | 54 | 1 | 460 bricks | One worker per team; cannot work during delivery |
| Team 2 – Set Sorting | Sort bricks into Axis, Chassis, and Accessories sets | B | C1, C2, C3 | 44 | 10 | 54 | 1 | 92 bricks | Simplified assumption: 1 brick = 1 sec work time |
| Team 3 – Axis Assembly | Assemble the Axis | C1 | D1 | 13 | 10 | 23 | 1 | 92 bricks | — |
| Team 4 – Chassis Assembly | Assemble the Chassis | C2 | D2 | 21 | 10 | 31 | 1 | 92 bricks | — |
| Team 5 – Final Assembly | Combine Axis and Chassis with accessories | C3, D1, D2 | E | 10 | 10 | 20 | 1 | 92 bricks | — |
| Seller | Sell the glider | E | F | — | — | — | — | — | Marks completion of process |

---

## Key Assumptions
- **Deterministic environment:** all processing times are fixed (no randomness in Sprint-1).
- **One worker per team:** workers cannot process and deliver simultaneously.
- **Batch size = 1:** each station handles one unit at a time.
- **Push production strategy:** stations start work as soon as materials are available.
- **No switching cost** or setup time for color changes (to be added in later sprints).
- **Simplified timing rule:** 1 brick = 1 second of work time.

---

## Purpose of Environment File
`environment.py` will:
1. Store all parameter values and object lists.
2. Define the connections between stations (flow rules).
3. Provide global constants to initialize the simulation.